In [37]:
from lark import Lark, Token

parser = Lark(r"""
               line : expr+

               expr : "(" expr* ")"
                    | "[" expr* "]"
                    | "{" expr* "}"
                    | "<" expr* ">"
               """,
               start='line',parser='lalr')

tmap = {'RPAR' : ')', 'RSQB': ']', 'RBRACE' : '}', 'MORETHAN' : '>'}
points = {'RPAR' : (3,1), 'RSQB': (57,2), 'RBRACE' : (1197,3), 'MORETHAN' : (25137,4)}
p = [0, []]

def auto_complete(e,score):
    next_token = next(t for t in e.interactive_parser.accepts() if t in points.keys())
    score = 5*score + points[next_token][1]
    next_token = Token(next_token,tmap[next_token])
    
    try:
        e.interactive_parser.feed_token(next_token)
        e.interactive_parser.feed_token(e.token)
    except Exception:
        score = auto_complete(e,score)

    return score

def error_handler(e):
    if e.token.type in points.keys(): 
        p[0] += points[e.token.type][0]
    else:
        p[1].append(auto_complete(e,0))

for line in open("input.txt").read().splitlines():
    try:
        parser.parse(line, on_error=error_handler)
    except:
        pass

print(p[0],sorted(p[1])[int(len(p[1])/2)])

464991 3662008566
